In [1]:
from pathlib import Path
import sys

root_dir = Path().resolve().parent.parent.parent.as_posix()
if root_dir not in sys.path:
    sys.path.append(root_dir)
del root_dir

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

from utils import config, datasets, models

In [2]:
MODEL_NAME = 'no_base_model-functional'
EPOCHS = 200

In [3]:
train_dataset, validation_dataset, test_dataset = datasets.load_dataset()

Found 6400 files belonging to 4 classes.


In [4]:
preprocessing = keras.Sequential([
    layers.Rescaling(1. / 255),
])

In [5]:
input_ = keras.layers.Input(shape=tf.TensorShape([128, 128, 3]), batch_size=32)

conv1_1 = layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same',
                        activation=keras.activations.relu)(input_)
conv1_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(input_)
conv1_5 = layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same',
                        activation=keras.activations.relu)(input_)

concatenate_1 = layers.Concatenate()([conv1_1, conv1_3, conv1_5])
pool_1 = layers.MaxPooling2D()(concatenate_1)

conv2_3 = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(pool_1)
conv2_5 = layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                        activation=keras.activations.relu)(pool_1)

concatenate_2 = layers.Concatenate()([conv2_3, conv2_5])
pool_2 = layers.MaxPooling2D()(concatenate_2)

dropout_2 = layers.Dropout(0.2)(pool_2)

conv3_3 = layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(dropout_2)

pool_3 = layers.MaxPooling2D()(conv3_3)
dropout_3 = layers.Dropout(0.25)(pool_3)

flatten = layers.Flatten()(dropout_3)

dense_1 = layers.Dense(256, activation=keras.activations.relu,
                       kernel_regularizer=keras.regularizers.l2(0.001))(flatten)
dense_2 = layers.Dense(256, activation=keras.activations.relu,
                       kernel_regularizer=keras.regularizers.l2(0.001))(dense_1)
output = layers.Dense(4, activation=keras.activations.softmax)(dense_2)

top = keras.Model(inputs=input_, outputs=output)

In [6]:
model = models.create_model(
    model_name=MODEL_NAME,
    preprocessing_layers=preprocessing,
    base_model=None,
    top_layers=top,
    optimizer=keras.optimizers.Adam()
)

In [ ]:
history = models.fit_model(
    model,
    train_data=train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

Epoch 1/200
160/160 [==============================] - 15s 78ms/step - loss: 1.4096 - sparse_categorical_accuracy: 0.4924 - val_loss: 1.0950 - val_sparse_categorical_accuracy: 0.5344
Epoch 2/200
160/160 [==============================] - 12s 76ms/step - loss: 1.0569 - sparse_categorical_accuracy: 0.5768 - val_loss: 0.9757 - val_sparse_categorical_accuracy: 0.5906
Epoch 3/200
160/160 [==============================] - 12s 77ms/step - loss: 0.9516 - sparse_categorical_accuracy: 0.6199 - val_loss: 0.8860 - val_sparse_categorical_accuracy: 0.6484
Epoch 4/200
160/160 [==============================] - 12s 77ms/step - loss: 0.8607 - sparse_categorical_accuracy: 0.6637 - val_loss: 0.8228 - val_sparse_categorical_accuracy: 0.6719
Epoch 5/200
160/160 [==============================] - 12s 76ms/step - loss: 0.7554 - sparse_categorical_accuracy: 0.7207 - val_loss: 0.6417 - val_sparse_categorical_accuracy: 0.7797
Epoch 6/200
160/160 [==============================] - 12s 76ms/step - loss: 0.6487 -

160/160 [==============================] - 12s 76ms/step - loss: 0.1790 - sparse_categorical_accuracy: 0.9830 - val_loss: 0.1720 - val_sparse_categorical_accuracy: 0.9875
Epoch 46/200
160/160 [==============================] - 12s 75ms/step - loss: 0.1835 - sparse_categorical_accuracy: 0.9818 - val_loss: 0.1772 - val_sparse_categorical_accuracy: 0.9828
Epoch 47/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1660 - sparse_categorical_accuracy: 0.9865 - val_loss: 0.1903 - val_sparse_categorical_accuracy: 0.9766
Epoch 48/200
160/160 [==============================] - 12s 75ms/step - loss: 0.1710 - sparse_categorical_accuracy: 0.9850 - val_loss: 0.1898 - val_sparse_categorical_accuracy: 0.9750
Epoch 49/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1648 - sparse_categorical_accuracy: 0.9887 - val_loss: 0.1832 - val_sparse_categorical_accuracy: 0.9812
Epoch 50/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1609 - sparse

160/160 [==============================] - 12s 75ms/step - loss: 0.1402 - sparse_categorical_accuracy: 0.9898 - val_loss: 0.1431 - val_sparse_categorical_accuracy: 0.9906
Epoch 90/200
160/160 [==============================] - 12s 75ms/step - loss: 0.1471 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.1440 - val_sparse_categorical_accuracy: 0.9937
Epoch 91/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1404 - sparse_categorical_accuracy: 0.9904 - val_loss: 0.1498 - val_sparse_categorical_accuracy: 0.9906
Epoch 92/200
160/160 [==============================] - 12s 75ms/step - loss: 0.1424 - sparse_categorical_accuracy: 0.9893 - val_loss: 0.1418 - val_sparse_categorical_accuracy: 0.9875
Epoch 93/200
160/160 [==============================] - 12s 75ms/step - loss: 0.1391 - sparse_categorical_accuracy: 0.9904 - val_loss: 0.1358 - val_sparse_categorical_accuracy: 0.9937
Epoch 94/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1376 - sparse

160/160 [==============================] - 12s 76ms/step - loss: 0.1398 - sparse_categorical_accuracy: 0.9879 - val_loss: 0.1440 - val_sparse_categorical_accuracy: 0.9859
Epoch 134/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1385 - sparse_categorical_accuracy: 0.9895 - val_loss: 0.1421 - val_sparse_categorical_accuracy: 0.9906
Epoch 135/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1264 - sparse_categorical_accuracy: 0.9910 - val_loss: 0.1338 - val_sparse_categorical_accuracy: 0.9906
Epoch 136/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1151 - sparse_categorical_accuracy: 0.9937 - val_loss: 0.1210 - val_sparse_categorical_accuracy: 0.9937
Epoch 137/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9932 - val_loss: 0.1179 - val_sparse_categorical_accuracy: 0.9953
Epoch 138/200
160/160 [==============================] - 12s 76ms/step - loss: 0.1267 - s

In [ ]:
model.summary()